In [6]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):
    h,w=K.shape#h,w是卷积核的高度和宽度
    Y=torch.zeros(X.shape[0]-h+1,X.shape[1]-h+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]= (X[i:i+h,j:j+w]*K).sum()
    return Y

### 与多层感知机的训练思路相同 卷积层被训练的参数是卷积核权重和标量偏置

In [3]:
class Conv2d(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))#weight是矩阵
        self.bias=nn.Parameter(torch.zeros(1))
        
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [4]:
#构造一个6*8像素的黑白图像。中间四列为黑色0，其余像素为白色1
x=torch.ones((6,8))
x[:,2:6]=0
x

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
#构造高度为1、宽度为2的卷积核K
K=torch.tensor([[1.0,-1.0]])   #构造思路 水平相邻的两元素相同，输出为0，否则输出为非零
Y=corr2d(x,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.,  0.]])

In [10]:
def comp_conv2d(conv2d,X):
    X=X.reshape((1,1)+X.shape)
    Y=conv2d(X)
    return Y.reshape(Y.shape[2:])#??
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1) #(3-1)/2
X= torch.rand(size=(8,8))
comp_conv2d(conv2d,X).shape


torch.Size([8, 8])

In [11]:
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [14]:
def corr2d_multi_in(X,k):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [15]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

## 1 * 1卷积层

In [16]:
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w=X.shape
    c_o=K.shape[0]    ##卷积输出通道数为2
    X=X.reshape((c_i,h*w))
    K=K.reshape((c_o,c_i))
    Y=torch.matmul(K,X)
    return Y.reshape((c_o,h,w))

In [18]:
X= torch.normal(0,1,(3,3,3))
K= torch.normal(0,1,(2,3,1,1))
X

tensor([[[-0.2790,  0.4501,  1.0748],
         [-1.1810, -0.4813, -1.1517],
         [-0.9784, -0.2526, -0.0867]],

        [[ 1.5539,  0.1029, -0.4231],
         [ 0.2723,  0.8079,  0.7693],
         [ 1.0240, -1.9154, -1.5742]],

        [[ 0.4208,  1.1938,  0.6759],
         [-0.0355,  0.4763,  0.0606],
         [-0.2947,  0.5205,  0.3657]]])

In [19]:
K

tensor([[[[ 0.3081]],

         [[-0.1402]],

         [[ 0.5605]]],


        [[[ 1.1649]],

         [[ 0.3740]],

         [[-1.2891]]]])

In [21]:
K.reshape((K.shape[0],X.shape[0]))

tensor([[ 0.3081, -0.1402,  0.5605],
        [ 1.1649,  0.3740, -1.2891]])

In [22]:
X.reshape((X.shape[0],X.shape[1]*X.shape[2]))

tensor([[-0.2790,  0.4501,  1.0748, -1.1810, -0.4813, -1.1517, -0.9784, -0.2526,
         -0.0867],
        [ 1.5539,  0.1029, -0.4231,  0.2723,  0.8079,  0.7693,  1.0240, -1.9154,
         -1.5742],
        [ 0.4208,  1.1938,  0.6759, -0.0355,  0.4763,  0.0606, -0.2947,  0.5205,
          0.3657]])